In [43]:
#

#1, 1, 2, ... , Fn+1 = Fn + Fn-1
#           2  1  1

#Fn+1 = 2 Fn [2^n] 
#Fn+1 = 2 Fn-1 [2^{n/2}]

#1, 2, 3, 4 ... [n]

def Fibonacci_num (n):
    if (n == 0 or n == 1):
        return 1
    
    return Fibonacci_num (n - 1) + Fibonacci_num (n - 2)

for i in range (10):
    print (i, Fibonacci_num (i))

0 1
1 1
2 2
3 3
4 5
5 8
6 13
7 21
8 34
9 55


In [1]:
import cv2
import numpy as np
import time
import math

WIND_X = 1200
WIND_Y = 700

class Branch:
    def __init__(self, length_, angle_, color_, angle_range_,
                 length_decrement_factor_, max_depth_, branching_factor_):
        self.length = length_ #+ np.random.randint (int (length_ / 3)) - \
                             #int (length_ / 6)
        self.angle  = angle_ + float (np.random.randint (50)) / 10000
        
        self.color  = color_
        
        #if (max_depth_ < 1):
        #    self.color = (100, 230, 230)
        
        self.children = []
        
        if (max_depth_ > 0):
            angle_step = angle_range_ / (branching_factor_ - 1)
            
            for i in range (branching_factor_):
                self.add_child (length_ = int (self.length * length_decrement_factor_),
                                angle_  = - angle_range_ / 2 +
                                    self.angle + i * angle_step,
                                color_  = self.color,
                                angle_range_ = angle_range_,
                                length_decrement_factor_ = length_decrement_factor_,
                                max_depth_ = max_depth_ - 1,
                                branching_factor_ = branching_factor_)

    def add_child (self, length_, angle_, color_, angle_range_,
                   length_decrement_factor_, max_depth_, branching_factor_):
        new_branch = Branch (length_, angle_, color_, angle_range_,
                             length_decrement_factor_,
                             max_depth_, branching_factor_)
        
        self.children.append (new_branch)
    
    def draw (self, img, x, y):
        x1 = int (x + self.length * math.cos (self.angle))
        y1 = int (y + self.length * math.sin (self.angle))

        cv2.line (img, (x, y), (x1, y1), self.color, 1)
        cv2.circle (img, (int (x1), int (y1)), 3, self.color, -1)

        for child in self.children:
            child.draw (img, x1, y1)

canvas_ = np.zeros ((WIND_Y, WIND_X, 3), np.uint8) * 35
canvas = canvas_.copy ()

while (True):
    root = Branch (length_ = 250, angle_ = -1.57, color_ = (160, 250, 110),
                   angle_range_ = 2.21, length_decrement_factor_ = 0.72,
                   max_depth_ = 5, branching_factor_ = 2)

    root.draw (canvas, int (WIND_X / 2), 700)

    cv2.imshow ("render", canvas)
    
    shadow = canvas.astype ("float") * 0.87
    
    new_canvas = canvas_.copy ()
    
    shift = 10
    new_canvas [:, shift :, :] = shadow [:, : -shift, :]
    
    canvas = new_canvas

    time.sleep (0.05)
    
    key = cv2.waitKey (1) & 0xFF
    
    if (key == ord ('q')):
        break
    
    #break
    
cv2.destroyAllWindows ()

In [ ]:
import numpy as np
import cv2
import math
from tqdm import tqdm

WIND_X, WIND_Y = 900, 600

def Mandelbrot_fractal (x, y, xsz, img):
    sh = img.shape

    for i in tqdm (range (sh [1])):
        for j in range (sh [0]):
            #x0 = (i - sh [1] / 3 * 2) / 250
            #y0 = (j - sh [0] / 2) / 250
            x0 = (i - sh [1] / 7 * 12) / 1550
            y0 = (j - sh [0] / 7 * 6) / 1550

            x, y = 0.0, 0.0

            iteration, max_iteration = 0, 200

            while (x**2 + y**2 <= 2*2 and iteration < max_iteration):
                xtemp = x**2 - y**2 + x0
                y = 2 * x * y + y0
                x = xtemp
                iteration += 1

            if (iteration < 199):
                img [j, i, :] = ((240 + int (70 * math.log2 (iteration))) % 255,
                                 (100 + int (50 * math.log2 (iteration))) % 255,
                                 (50  + int (0)) % 255)

    return img

frame = Mandelbrot_fractal (1, 0, 2, np.zeros ((WIND_Y, WIND_X, 3), np.uint8))

cv2.imshow ("render", frame)

cv2.waitKey (0)

100%|██████████| 900/900 [03:51<00:00,  2.90it/s]
